In [ ]:
!git clone https://github.com/openai/shap-e
%cd shap-e
!pip install -e .

Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 336 (delta 41), reused 218 (delta 18), pack-reused 76
Receiving objects: 100% (336/336), 11.72 MiB | 15.14 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/content/shap-e
Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-uz31sp1e/clip_4952cb3b4efa4dbaa70d80300d5702b6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-uz31sp1e/clip_4952cb3b4efa4dbaa70d80300d5702b6
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.

In [ ]:
import torch
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
import warnings
warnings.filterwarnings("ignore", message="A NumPy version .* is required for this version of SciPy")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:09<00:00, 98.3MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [ ]:
# you can change render_mode_ to 'stf'or 'nerf'
# size = size of the renders --> higher values take longer to render

def generate_3d(prompt_, size_=64, render_mode_ ='nerf'):
    batch_size = 1
    guidance_scale = 15.0
    prompt = prompt_

    latents = sample_latents(
        batch_size=batch_size,
        model=model,
        diffusion=diffusion,
        guidance_scale=guidance_scale,
        model_kwargs=dict(texts=[prompt] * batch_size),
        progress=True,
        clip_denoised=True,
        use_fp16=True,
        use_karras=True,
        karras_steps=64,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0)
    render_mode = render_mode_
    size = size_
    cameras = create_pan_cameras(size, device)
    for i, latent in enumerate(latents):
        images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
        display(gif_widget(images))


In [ ]:
generate_3d('an airplane that looks like a banana', 256)

  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhAAEAAYcAAPb0APPzAPPyAPLyAPLxAPHxAPDxAPLwAPDwAO/wAPDvAO/vAO…

In [ ]:
generate_3d('A smart Phone',256)



  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhAAEAAYcAAOjp6ejo6ejo6Ojn6Ofn6Ofn5+fm5+bn5+bm5+bm5uXm5ubl5u…

In [ ]:
generate_3d('A smart Phone',256,render_mode_='stf')



  0%|          | 0/64 [00:00<?, ?it/s]

/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


HTML(value='<img src="data:image/gif;base64,R0lGODlhAAEAAYcAAOzs6+Hg4ODg4ODg3+Df4ODf39/f39/f3t/e39/e3t7e3t7e3d…